In [56]:
# Option simil RASA NLU intent classification
# Data preparation
import spacy
from sklearn.preprocessing import LabelEncoder

# NOTE: we should take advantage of the split_train_test function here, this is just a simple example
# Define the main corpus
queries = ['hi', 'hello', 'morning', 'bye', 'see you!', 'Suggest me some mexican restaurants', 'im looking for restaurants']
labels = ['greet', 'greet', 'greet', 'end_conversation', 'end_conversation', 'restaurant_search', 'restaurant_search']

queries_test = ['hi there', 'goodbye', 'is there any chinese restaurants?']
labels_test = ['greet',  'end_conversation',  'restaurant_search']

# Tokenize the words for TRAIN
nlp = spacy.load('en_core_web_lg')

n_queries=len(queries)
dim_embedding = nlp.vocab.vectors_length
X = np.zeros((n_queries, dim_embedding))

for idx, sentence in enumerate(queries):
   doc = nlp(str(sentence))
   X[idx, :] = doc.vector

# Encode labels
le = LabelEncoder()
y = le.fit_transform(labels)

# --------------------------------------------

# Tokenize the words for TEST
n_queries_test=len(queries_test)
X_test = np.zeros((n_queries_test, dim_embedding))


for idx, sentence in enumerate(queries_test):
   doc = nlp(str(sentence))
   X_test[idx, :] = doc.vector

# Encode labels
y_test = le.fit_transform(labels_test)
print(y.shape, X.shape, y_test.shape, X_test.shape)

(7,) (7, 300) (3,) (3, 300)


In [57]:
# Search for the best model
import numpy as np
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

defaults = {                              
    "C": [1, 2, 5, 10, 20, 100],
    "kernels": ["linear"], "max_cross_validation_folds": 5
}

C = defaults["C"]                       
kernels = defaults["kernels"]

tuned_parameters = [{"C": C, "kernel": [str(k) for k in kernels]}]                                                                 
folds = defaults["max_cross_validation_folds"]
cv_splits = max(2, min(folds, np.min(np.bincount(y)) // 5))

svc = svm.SVC(C=1, probability=True, class_weight='balanced')
clf = GridSearchCV( 
    svc,
    param_grid=tuned_parameters, n_jobs=1, cv=cv_splits,         
    scoring='f1_weighted', verbose=1)
clf.fit(X, y)

print(sorted(clf.cv_results_.keys()))
print("Accuracy on Test Set: ",np.count_nonzero(clf.predict(X_test)==y_test)/len(y_test))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
['mean_fit_time', 'mean_score_time', 'mean_test_score', 'param_C', 'param_kernel', 'params', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'std_fit_time', 'std_score_time', 'std_test_score']
Accuracy on Test Set:  1.0


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [64]:
# Intent classification

X_test = np.array(nlp('hi there').vector).reshape(1, -1)                   
pred_result = clf.predict(X_test)
print(pred_result)

le.inverse_transform(pred_result)

[1]


array(['greet'], dtype='<U17')

In [72]:
# Extract Entities

# Option 1 - Use spaCy, it should be sufficient to what we want to do
# but for sure we are going to need to train it better for improved outcome --> https://spacy.io/usage/training#training-data
# In the example below you can see that it did not recognize restaurant as a NOUN
doc = nlp('Suggest me some mexican restaurant')

for ent in doc.ents:
    print(ent, ent.label_)


# Option 2 - A specialized-recognizer by using sklearn https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html

mexican NORP
